In [1]:
import pickle

In [165]:
import numpy
import scipy
import pandas
import spacy
import textacy

In [171]:
k = numpy.int()

In [3]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

In [177]:
with open('/home/immersinn/Dropbox/Analytics/NCGA/data/bill_texts_filed_content.pkl', 'rb') as f1:
    btfc = pickle.load(f1)

In [178]:
btfc = btfc.set_index(["session", "house", "bill"])

In [179]:
btfc.head()

content  \
session house bill                                                      
2015E4  H     1     GENERAL ASSEMBLY OF NORTH CAROLINA  FOURTH EXT...   
              2     GENERAL ASSEMBLY OF NORTH CAROLINA  FOURTH EXT...   
              3     GENERAL ASSEMBLY OF NORTH CAROLINA  FOURTH EXT...   
              4     GENERAL ASSEMBLY OF NORTH CAROLINA  FOURTH EXT...   
              5     GENERAL ASSEMBLY OF NORTH CAROLINA  FOURTH EXT...   

                                                           long_title  \
session house bill                                                      
2015E4  H     1     A HOUSE RESOLUTION adopting the permanent rule...   
              2     A JOINT RESOLUTIOn providing for adjournment s...   
              3     AN ACT to provide further REGULATORY RELIEF TO...   
              4     AN ACT directing the department of transportat...   
              5     AN ACT to clarify the service area for communi...   

                                                           table_info  
session house bill                                                     
2015E4  H     1     {'Referred to': '', 'Sponsors': 'Representativ...  
              2     {'Referred to': '', 'Sponsors': 'Representativ...  
              3     {'Short   Title': 'Regulatory Reform Act of 20...  
              4     {'Short   Title': 'Terminate Agreement for Tol...  
              5     {'Short   Title': 'Municipal Broadband Service...

In [180]:
with open('/home/immersinn/Dropbox/Analytics/NCGA/data/bill_page_keywords.pkl', 'rb') as f1:
    keywords = pickle.load(f1)

In [181]:
keywords = keywords.set_index(["session", "house", "bill"])

In [182]:
keywords.head()

keywords
session house bill                                                   
2015E4  H     1     [ADOPTED, GENERAL ASSEMBLY, RESOLUTIONS, SIMPL...
              2     [ADJOURNMENT, GENERAL ASSEMBLY, RESOLUTIONS, J...
              3     [ADMINISTRATION DEPT., ADMINISTRATIVE CODE, AD...
              4     [BRIDGES, CONTRACTS, COUNTIES, INFRASTRUCTURE,...
              5     [COUNTIES, EDGECOMBE COUNTY, INFORMATION TECHN...

In [183]:
btfc = btfc.join(keywords, how="left")

In [184]:
btfc = pandas.DataFrame(btfc.to_records())

In [185]:
btfc.head()

,session,house,bill,content,long_title,table_info,keywords
0,2015E4,H,1,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,A HOUSE RESOLUTION adopting the permanent rule...,"{'Referred to': '', 'Sponsors': 'Representativ...","[ADOPTED, GENERAL ASSEMBLY, RESOLUTIONS, SIMPL..."
1,2015E4,H,2,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,A JOINT RESOLUTIOn providing for adjournment s...,"{'Referred to': '', 'Sponsors': 'Representativ...","[ADJOURNMENT, GENERAL ASSEMBLY, RESOLUTIONS, J..."
2,2015E4,H,3,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,AN ACT to provide further REGULATORY RELIEF TO...,{'Short Title': 'Regulatory Reform Act of 20...,"[ADMINISTRATION DEPT., ADMINISTRATIVE CODE, AD..."
3,2015E4,H,4,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,AN ACT directing the department of transportat...,{'Short Title': 'Terminate Agreement for Tol...,"[BRIDGES, CONTRACTS, COUNTIES, INFRASTRUCTURE,..."
4,2015E4,H,5,GENERAL ASSEMBLY OF NORTH CAROLINA FOURTH EXT...,AN ACT to clarify the service area for communi...,{'Short Title': 'Municipal Broadband Service...,"[COUNTIES, EDGECOMBE COUNTY, INFORMATION TECHN..."


In [189]:
btfc._get_numeric_data().columns.tolist()

['bill']

In [135]:
btfc['bill'] = btfc.apply(lambda x: str(x['bill']), axis=1)

In [152]:
btfc_mini = btfc.sample(400)

In [155]:
btfc_mini.index = range(btfc_mini.shape[0])

## Textual Analysis

### Preproc and Build Corpus

In [20]:
textacy_preprocessor = lambda text: textacy.preprocess.preprocess_text(text,
                                                                       no_contractions=True,
                                                                       no_numbers=True,
                                                                       no_emails=True,
                                                                       no_currency_symbols=True,
                                                                       lowercase=True)

In [21]:
nlp = spacy.load("en", add_vectors=False)

In [22]:
nlp.pipeline = [nlp.tagger, nlp.parser]

In [153]:
# "texts"
texts = [textacy_preprocessor(con) for con in list(btfc_mini.content)]

In [156]:
metas = [btfc_mini.ix[btfc.index[i]][["session", "house", "bill", "keywords"]].to_dict() for i in range(btfc_mini.shape[0])]

In [157]:
metas[0]

{'bill': '660',
 'house': 'H',
 'keywords': ['BOARDS',
  'COMMISSIONS',
  'CONTRACTS',
  'EDUCATION',
  'EDUCATION ADMINISTRATION',
  'EDUCATION',
  'STATE BOARD OF',
  'ELECTRONIC & OTHER MEDIA',
  'ELEMENTARY EDUCATION',
  'EMERGING TECHNOLOGIES',
  'GOVERNMENT EMPLOYEES',
  'OCCUPATIONS',
  'PUBLIC',
  'REPORTING',
  'SCIENCE & TECHNOLOGY',
  'SECONDARY EDUCATION',
  'STATE EMPLOYEES',
  'TEACHERS',
  'TEXTBOOK COMN.',
  'TEXTBOOKS'],
 'session': '2015'}

In [158]:
corp = textacy.Corpus(lang=nlp)

In [159]:
corp.add_texts(texts=texts, metadatas=metas,
               n_threads=8, batch_size=50)

In [160]:
corp.n_docs

400

In [161]:
corp.n_sents

22988

In [162]:
corp[0].metadata

{'bill': '660',
 'house': 'H',
 'keywords': ['BOARDS',
  'COMMISSIONS',
  'CONTRACTS',
  'EDUCATION',
  'EDUCATION ADMINISTRATION',
  'EDUCATION',
  'STATE BOARD OF',
  'ELECTRONIC & OTHER MEDIA',
  'ELEMENTARY EDUCATION',
  'EMERGING TECHNOLOGIES',
  'GOVERNMENT EMPLOYEES',
  'OCCUPATIONS',
  'PUBLIC',
  'REPORTING',
  'SCIENCE & TECHNOLOGY',
  'SECONDARY EDUCATION',
  'STATE EMPLOYEES',
  'TEACHERS',
  'TEXTBOOK COMN.',
  'TEXTBOOKS'],
 'session': '2015'}

In [163]:
corp.save(path='/home/immersinn/gits/ncga/data/processed/', name='CORPUS_bills_filed_pipe01', compression='gzip')

In [164]:
corp = textacy.Corpus.load(path='/home/immersinn/gits/ncga/data/processed/',
                           name='CORPUS_bills_filed_pipe01',
                           compression='gzip')

### Topic Model

In [72]:
n_topics = 15

#### Tokenize and Vectorize the corpus:

In [46]:
terms_lists = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True) for doc in corp)
doc_term_matrix, id2term = textacy.vsm.doc_term_matrix(\
                                                       terms_lists, 
                                                       weighting='tfidf', normalize=True, smooth_idf=True,
                                                       min_df=3, max_df=0.95, max_n_terms=100000)
doc_term_matrix

<2098x9955 sparse matrix of type '<class 'numpy.float64'>'
	with 378044 stored elements in Compressed Sparse Row format>

#### Init and Train a Topic Model:

In [73]:
model = textacy.tm.TopicModel('nmf', n_topics=n_topics)
model.fit(doc_term_matrix)
model

TopicModel(n_topics=15, model=NMF)

#### Transform the corpus and Interpret the Model:

In [77]:
doc_topic_matrix = model.transform(doc_term_matrix)

In [74]:
for topic_idx, top_terms in model.top_topic_terms(id2term, topics=range(5)):
    print('topic', topic_idx, ':', '   '.join(top_terms))

topic 0 : shall   state   commission   department   agency   information   contract   include   require   article
topic 1 : school   close   opening   calendar   date   closing   board   local   monday   august
topic 2 : senatorial   district   act-*number*.   relate   local   senator   senate   law   primary   short
topic 3 : tax   taxable   taxpayer   income   credit   sale   year   levy   property   business
topic 4 : fund   fiscal   appropriation   usd*number   year   dollar   appropriate   thousand   budget   highway


In [84]:
for topic_idx, top_docs in model.top_topic_docs(doc_topic_matrix, topics=[0,1,2,3], top_n=2):
    print(topic_idx)
    for j in top_docs:
        print(corp[j].metadata['keywords'])

0
['ADMINISTRATION DEPT.', 'ADMINISTRATIVE CODE', 'ADMINISTRATIVE RULES', 'AGRICULTURAL LAND', 'AGRICULTURE', 'AGRICULTURE COMMISSIONER', 'AGRICULTURE DEPT.', 'ANIMALS', 'APPOINTMENTS', 'ARMED FORCES', 'BEACH EROSION', 'BOARDS', 'BUDGET & MANAGEMENT OFFICE', 'BUILDING CODE COUNCIL', 'BUILDING CODES', 'CHIEF INFORMATION OFFICER', 'COASTAL RESOURCES', 'COASTAL RESOURCES COMN.', 'COLLEGES & UNIVERSITIES', 'COMMERCE', 'COMMISSIONS', 'COMMITTEES', 'COMMUNITY COLLEGES', 'COMMUNITY COLLEGES OFFICE', 'CONSERVATION', 'CONSTRUCTION', 'CONTRACTS', 'CORPORATIONS', 'NONPROFIT', 'COUNCIL OF STATE', 'COUNCILS', 'COUNTIES', 'CULTURAL RESOURCES', 'DAIRY PRODUCTS', 'DATA SYSTEMS', 'DEFINITIONS', 'DEQ', 'DMV', 'EDUCATION', 'EDUCATION BOARDS', 'ELECTRONIC GOVERNMENT', 'ELEMENTARY EDUCATION', 'EMPLOYMENT', 'ENVIRONMENT', 'ENVIRONMENTAL MANAGEMENT COMN.', 'EROSION STRUCTURES', 'FEES', 'FINANCIAL SERVICES', 'FISH & WILDLIFE', 'FISHERIES', 'FOODS & BEVERAGES', 'FRANCHISES', 'GENERAL ASSEMBLY', 'GENERAL CONTRA

#### View Top Terms, Top Keywords from Top Documents

In [101]:
pandas.Series(doc_topic_matrix.reshape((doc_topic_matrix.shape[0]*doc_topic_matrix.shape[1],))).describe(percentiles=[0.75, 0.80, 0.90, 0.95, 0.975, 0.99])

count    31470.000000
mean         0.011770
std          0.035995
min          0.000000
50%          0.000000
75%          0.001839
80%          0.007263
90%          0.034381
95%          0.073173
97.5%        0.122555
99%          0.190016
max          0.399531
dtype: float64

In [102]:
sum(doc_topic_matrix > 0.1)

array([ 83,  52,  50,  92,  67, 100,  82,  78,  72,  96,  68,  67,  52,
        25,  71])

In [85]:
top_term_table = []
index = []
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n = 15, topics=range(n_topics)):
    index.append('topic ' + str(topic_idx))
    top_term_table.append({i : tt for i,tt in enumerate(top_terms)})
top_term_table = pandas.DataFrame(data=top_term_table, index=index)

In [86]:
top_term_table.transpose()

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,topic 11,topic 12,topic 13,topic 14
0,shall,school,senatorial,tax,fund,person,health,employee,resolution,school,vehicle,election,town,state,member
1,state,close,district,taxable,fiscal,court,child,employer,committee,student,plate,voter,property,united,board
2,commission,opening,act-*number*.,taxpayer,appropriation,offense,service,wage,senate,education,motor,vote,city,constitution,appoint
3,department,calendar,relate,income,usd*number,defendant,care,retirement,joint,charter,registration,candidate,council,convention,shall
4,agency,date,local,credit,year,shall,medicaid,plan,house,teacher,license,shall,corporate,amendment,term
5,information,closing,senator,sale,dollar,felony,program,tip,legislative,grade,division,county,charter,congress,trustee
6,contract,board,senate,year,appropriate,judge,human,benefit,representative,board,driver,board,lien,legislature,governor
7,include,local,law,levy,thousand,petitioner,plan,employment,rule,program,fee,ballot,limit,v,serve
8,require,monday,primary,property,budget,violation,department,state,bill,shall,special,office,county,propose,education
9,article,august,short,business,highway,order,division,leave,extra,local,dealer,elect,describe,america,appointment


In [93]:
top_keywords = {}
for topic_idx, top_docs in model.top_topic_docs(doc_topic_matrix, topics=range(n_topics), top_n=10):
    topic = 'topic ' + str(topic_idx)
    keywords = []
    for j in top_docs:
        keywords.extend(corp[j].metadata['keywords'])
    top_keywords[topic] = set(keywords)

In [94]:
for topic_idx in range(n_topics):
    k = "topic " + str(topic_idx)
    print("\n")
    print(k + ":")
    print(top_keywords[k])



topic 0:
{'ANIMALS', 'LAKES & RIVERS', 'LANDOWNERS', 'INFORMATION TECHNOLOGY', 'APPOINTMENTS', 'DMV', 'PLANNING & ZONING', 'CHIEF INFORMATION OFFICER', 'PROGRAM EVALUATION DIVISION', 'SECONDARY EDUCATION', 'ELECTRICITY', 'TRANSPORTATION DEPT.', 'EDUCATION BOARDS', 'EMINENT DOMAIN', 'STATE BUILDINGS', 'PUBLIC OFFICIALS', 'LOANS', 'PRIVACY', 'CONTRACTS', 'RATIFIED', 'SPEED LIMIT', 'EROSION STRUCTURES', 'STATE EMPLOYEES', 'FOODS & BEVERAGES', 'HUNTING & FISHING', 'TITLE CHANGE', 'COUNCIL OF STATE', 'EMPLOYMENT', '911 BOARD', 'CHAPTERED', 'MUSEUMS', 'MANAGEMENT', 'CONSTRUCTION', 'NATURAL & CULTURAL RESOURCES DEPT.', 'SCHOOL OF GOVERNMENT', 'UNC', 'HORSES & MULES', 'LIVESTOCK & POULTRY', 'GOVERNMENT EMPLOYEES', 'BOARDS', 'STATUTE OF LIMITATIONS', 'PILOT PROGRAMS', 'COMMUNITY COLLEGES', 'LEASING', 'PUBLIC BUILDINGS', 'LOCAL ORDINANCES', 'ROADS & HIGHWAYS', 'MUNICIPALITIES', 'COLLEGES & UNIVERSITIES', 'VETERINARY MEDICINE', 'INFORMATION TECHNOLOGY DEPT.', 'UTILITIES COMN.', 'COMMITTEES', 'S